<a href="https://colab.research.google.com/github/Zveroloff/deepnlp_alisa/blob/master/Alice_in_Fridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

At first, just install DeepPavlov models and modules

In [2]:
!pip install deeppavlov

    100% |████████████████████████████████| 512kB 7.2MB/s 
    100% |████████████████████████████████| 4.1MB 8.5MB/s 
    100% |████████████████████████████████| 12.4MB 3.5MB/s 
    100% |████████████████████████████████| 11.8MB 4.7MB/s 
    100% |████████████████████████████████| 12.2MB 3.7MB/s 
    100% |████████████████████████████████| 51kB 15.8MB/s 
    100% |████████████████████████████████| 8.0MB 3.7MB/s 
    100% |████████████████████████████████| 51kB 22.0MB/s 
    100% |████████████████████████████████| 3.4MB 13.3MB/s 
    100% |████████████████████████████████| 307kB 25.1MB/s 
    100% |████████████████████████████████| 92kB 26.5MB/s 
    100% |████████████████████████████████| 61kB 27.5MB/s 
    100% |████████████████████████████████| 7.1MB 6.0MB/s 
    100% |████████████████████████████████| 51kB 21.7MB/s 
  Running setup.py bdist_wheel for overrides ... - done
  Stored in directory: /root/.cache/pip/wheels/8d/52/86/e5a83b1797e7d263b458d2334edd2704c78508b3eea9323718
  Ru

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!python -m deeppavlov install gobot_dstc2
!python -m deeppavlov download gobot_dstc2

2018-12-11 21:04:24.324 INFO in 'deeppavlov.core.common.file'['file'] at line 31: Interpreting 'gobot_dstc2' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/go_bot/gobot_dstc2.json'
    100% |████████████████████████████████| 58.4MB 762kB/s 
    100% |████████████████████████████████| 573kB 18.2MB/s 
    100% |████████████████████████████████| 3.3MB 9.7MB/s 
magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.10.0 which is incompatible.
google-colab 0.0.1a1 has requirement requests~=2.18.0, but you'll have requests 2.19.1 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: setuptools 40.6.2
    Uninstalling setuptools-40.6.2:
      Successfully uninstalled setuptools-40.6.2
  Found existing installation: tensorboard 1.12.0
    Uninstalling tensorboard-1.12.0:
      Successfully uninstalled tensorboard-1.12.0
  Found existing installation: ten

In [0]:
!python -m deeppavlov interact gobot_dstc2

Let's import Alice

In [0]:
import json
import logging


In [0]:
# Импортируем подмодули Flask для запуска веб-сервиса.
from flask import Flask, request
app = Flask(__name__)

logging.basicConfig(level=logging.DEBUG)

# Хранилище данных о сессиях.
sessionStorage = {}

# Задаем параметры приложения Flask.
@app.route("/", methods=['POST'])

In [0]:
def main():
# Функция получает тело запроса и возвращает ответ.
    logging.info('Request: %r', request.json)

    response = {
        "version": request.json['version'],
        "session": request.json['session'],
        "response": {
            "end_session": False
        }
    }

    handle_dialog(request.json, response)

    logging.info('Response: %r', response)

    return json.dumps(
        response,
        ensure_ascii=False,
        indent=2
    )

  

In [0]:
# Функция для непосредственной обработки диалога.
def handle_dialog(req, res):
    user_id = req['session']['user_id']

    if req['session']['new']:
        # Это новый пользователь.
        # Инициализируем сессию и поприветствуем его.

        sessionStorage[user_id] = {
            'suggests': [
                "Не хочу.",
                "Не буду.",
                "Отстань!",
            ]
        }

        res['response']['text'] = get_response(user_id)
        res['response']['buttons'] = get_suggests(user_id)
        return

    # Обрабатываем ответ пользователя.
    if req['request']['original_utterance'].lower() in [
        'выход'
    ]:
        # Пользователю надоело, прощаемся
        res['response']['text'] = 'До свидания!'
        return

    # Если нет, то убеждаем его купить слона!
    res['session']['original_utterance'] = req['request']['original_utterance'].lower()
    res['response']['buttons'] = get_suggests(user_id)


In [0]:
# Функция возвращает две подсказки для ответа.
def get_suggests(user_id):
    session = sessionStorage[user_id]

    # Выбираем две первые подсказки из массива.
    suggests = [
        {'title': suggest, 'hide': True}
        for suggest in session['suggests'][:2]
    ]

    # Убираем первую подсказку, чтобы подсказки менялись каждый раз.
    session['suggests'] = session['suggests'][1:]
    sessionStorage[user_id] = session

    # Если осталась только одна подсказка, предлагаем подсказку
    # со ссылкой на Яндекс.Маркет.
    if len(suggests) < 2:
        suggests.append({
            "title": "Ладно",
            "url": "https://market.yandex.ru/search?text=слон",
            "hide": True
        })

    return suggests

#Научим Алису говорить на языке Павлова

Для начала инициализируем простого агента:


In [0]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

hello = PatternMatchingSkill(responses=['Hello world!'], patterns=["hi", "hello", "good day"])
bye = PatternMatchingSkill(['Goodbye world!', 'See you around'],
                           patterns=["bye", "chao", "see you"])
fallback = PatternMatchingSkill(["I don't understand, sorry", 'I can say "Hello world!"'])

agent = DefaultAgent([hello, bye, fallback], skills_selector=HighestConfidenceSelector())

Теперь напишем функцию ответа

In [0]:
def get_response(user_id):
    session = sessionStorage[user_id]

    # Выбираем две первые подсказки из массива.
    responses = agent(['Hello'])

    # Убираем первую подсказку, чтобы подсказки менялись каждый раз.
    session['suggests'] = session['suggests'][1:]
    sessionStorage[user_id] = session

    return responses[0]